# Imports 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from deap import base, creator, tools

#file generated by us that has the functions created by us 
from auxfunctions import * 

# Read files 

In [2]:
CustDist_WHCentral = pd.read_csv("CustDist_WHCentral.csv", sep=",", decimal=".")
del CustDist_WHCentral["Distances between Customers and Warehouse"]
CustDist_WHCentral_list = CustDist_WHCentral.values.tolist()

CustDist_WHCorner = pd.read_csv("CustDist_WHCorner.csv", sep=",", decimal=".")
del CustDist_WHCorner["Distances between Customers and Warehouse"]
CustDist_WHCorner_list = CustDist_WHCorner.values.tolist()

CustOrd = pd.read_csv("CustOrd.csv", sep=",", decimal=".")
del CustOrd["Customer"]
CustOrd_list = CustOrd.values.tolist()

#  Change parameters of the execution of the program here 

### Fixed parameters 

In [3]:
#defined in the assignment 
Number_of_Evaluations = 10000

### Variable parameters 

In [57]:
#+1 to consider warehouse (options -> 51, 31, 16)
Number_Customers = 51     
#size of the population
Population_Size = 100

# CXPB  is the probability with which two individuals are crossed
CXPB = 0.9
# MUTPB is the probability for mutating an individual
MUTPB = 0.1

#comment the unwanted option to select the warehouse location
Warehouse_location = "Central"
#Warehouse_location = "Corner"

#comment the unwanted option to select the type of costs in deliveries
deliver_type = "variable"
#deliver_type = "fixed"


### Prints and number of iterations

In [46]:
#change to 1 if print you want prints 
prints = 2

#define the number of iterations 
n_iterations = 30

In [49]:
def P1():

    #selects the file to be used with the costs 
    if(Warehouse_location == "Central"):
        CustDist = CustDist_WHCentral_list
    else:
        CustDist = CustDist_WHCorner_list
     
    #selects the mode of the costs in deliveries 
    if(deliver_type == "variable"):
        CustOrd = CustOrd_list
    else: 
        CustOrd = None 

        
    #results is a list of (best_ind, mean, std, min)
    results = []
    
    #iterate over a previous defined number of iterations 
    for i in range(n_iterations):
        seed = random.randint(1, 100000) 
        results.append(EASingleObjective(Number_Customers, Population_Size, 
                                         Number_of_Evaluations, seed, CustDist, CustOrd))
    
    min_index = 0
    minimum = 10000
    avg_mean = 0
    avg_std = 0
    avg_min = 0
    #interpretate the results
    for i in range(len(results)):
        if(results[i][3] < minimum):
            min_index = 0
        avg_mean = avg_mean + results[i][1]
        avg_std = avg_std + results[i][2]
        avg_min = avg_min + results[i][3]
    
    avg_mean = avg_mean / n_iterations
    avg_std = avg_std / n_iterations
    avg_min = avg_min / n_iterations
    
    print("  At the end of ",n_iterations, " executions")
    print("  Avg Min %s" % avg_min)
    print("  Avg mean %s" % avg_mean)
    print("  Avg Std %s" % avg_std)
    print("  Best individual is %s, %s" % (results[min_index][0], results[min_index][3]))
    
    return 

In [55]:
def EASingleObjective(Number_Customers, Population_Size, Number_of_Evaluations, seed, CustDist, CustOrd_list):
    
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    ## permutation setup for individual,
    toolbox.register("indices", \
                     random.sample, \
                     range(Number_Customers), 
                     Number_Customers)

    toolbox.register("individual", \
                     tools.initIterate, \
                     creator.Individual, toolbox.indices)

    ## population setup,
    toolbox.register("population", \
                     tools.initRepeat, \
                     list, toolbox.individual)

    #setup of the methods used in evolution
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
    toolbox.register("select", tools.selTournament, tournsize=5)
    
    #definition of the evaluation function
    if(CustOrd_list != None):
        toolbox.register("evaluate", evaluatecostvariable)
    else:
        toolbox.register("evaluate", evaluatecostfixed)
        
    
    random.seed(seed)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=Population_Size)
    
    if(prints == 1):
        print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = []
    for ind in pop:
        fitnesses.append(toolbox.evaluate(ind, CustDist, CustOrd_list))
    #fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    if(prints == 1):    
        print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    arr_mean = []
    arr_min = []
    # Begin the evolution
    while g < Number_of_Evaluations/ Population_Size:
        # A new generation
        g = g + 1
        
        if(prints == 1):
            print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = []
        for ind in invalid_ind:
            fitnesses.append(toolbox.evaluate(ind, CustDist, CustOrd_list))
        #fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        if(prints == 1):
            print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        if(prints == 1):
            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)

        arr_mean.append(mean)
        arr_min.append(min(fits))
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    
    if(prints == 1):
        print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
        x_coordinate = [i+1 for i in range(len(arr_mean))]
        plt.plot(x_coordinate, arr_mean, label = "Average")
        plt.plot(x_coordinate, arr_min, label = "Min")
        plt.title("Min and Average Fitness over generations")
        plt.legend()
        plt.grid()
        plt.figure()

    return (best_ind, mean, std, min(fits))

In [58]:
P1()

-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolution --
-- End of (successful) evolu